In [2]:
import numpy as np
import pandas as pd
import pymongo
from bson.son import SON
from bson.code import Code
import os

# importazione da script di sistema
os.system('mongoimport -d titanic -c passengers --headerline --file ~/data/titanic.csv --type csv')

myclient = pymongo.MongoClient("mongodb://localhost:27017/") # 27017 è la porta di default

db = myclient["titanic"] # anche 'db = myclient.titanic'



2022-11-23T16:19:25.755+0100	connected to: mongodb://localhost/
2022-11-23T16:19:25.800+0100	891 document(s) imported successfully. 0 document(s) failed to import.


## Definizione del mapper, del reducer e del finalizer

E' importante che il reducer restituisca esattamente lo stesso oggetto che riceve come val cioè il value emesso dal mapper. Infatti il reducer può essere richiamato più volte su singole porzioni di dati mappati i quali devono essere quindi concatenati a quelli già processati in precedenza dal reducer stesso.

Il mapper emette un oggetto siffatto:

```javascript
{id_classe, {countMale: 1/0, countFemale: 1/0}}
```
a seconda che il passeggero sia maschio o femmina. 

Il reducer riduce l'array di oggetti emessi per la stessa classe generando un solo oggetto siffatto:

```javascript
{id_classe, {countMale: <somma uomini per classe>, countFemale: <somma donne per classe}}
```
Infine, il finalizer calcola le percentuali di sopravvivenza utilizzando lo scope che è costituito dalle query di conteggio dei passeggeri, maschi e femmine, per ogni classe.



In [3]:
mapfun = Code("""
    function (){

        var key = this.Pclass;
        var value = {countMale: 0, countFemale: 0};
        
        if (this.Sex == 'male')
            value.countMale=1;
        else
            value.countFemale=1;
        
        emit(key,value);
    }
""");


reducefun = Code("""
    function (key, val){
    
        var survived = {countMale: 0, countFemale: 0};
        
        for (var i=0; i < val.length; i++){
            survived.countMale+=val[i].countMale;
            survived.countFemale+=val[i].countFemale;
        }
        
        return survived;
            
    }
""");

finalizefun = Code("""
    function finalize(key, reducedval){
        
        if(key == 1.0){
            reducedval.countMale /= males1Class;
            reducedval.countFemale /= females1Class;
            }
        else if(key == 2.0){
            reducedval.countMale /= males2Class;
            reducedval.countFemale /= females2Class;
        }
        else {
            reducedval.countMale /= males3Class;
            reducedval.countFemale /= females3Class;
        }
        
        reducedval.countMale*=100;
        reducedval.countMale = reducedval.countMale.toPrecision(4)*1;
        reducedval.countFemale*=100;
        reducedval.countFemale= reducedval.countFemale.toPrecision(4)*1;

        return reducedval;
    }
""")

In [4]:
# richiediamo al database i valori globali degli imbarcati per genere e per classe
# per ottenere i tassi di sopravvivenza
# e non i valori assoluti dei sopravvissuti

males_1_class=db.passengers.count_documents({"Pclass": 1, "Sex": "male"})
females_1_class=db.passengers.count_documents({"Pclass": 1, "Sex": "female"})
males_2_class=db.passengers.count_documents({"Pclass": 2, "Sex": "male"})
females_2_class=db.passengers.count_documents({"Pclass": 2, "Sex": "female"})
males_3_class=db.passengers.count_documents({"Pclass": 3, "Sex": "male"})
females_3_class=db.passengers.count_documents({"Pclass": 3, "Sex": "female"})

mapreduce_info = db.passengers.map_reduce(mapfun,
                                          reducefun,
                                          "survivedPassengers",
                                          full_response=True, # restituisce le info di esecuzione
                                          query={"Survived":1},
                                          scope={"males1Class": males_1_class,
                                                 "females1Class": females_1_class,
                                                 "males2Class": males_2_class,
                                                 "females2Class": females_2_class,
                                                 "males3Class": males_3_class,
                                                 "females3Class": females_3_class},
                                          finalize=finalizefun)


/tmp/ipykernel_19366/3550242101.py:12: DeprecationWarning: map_reduce is deprecated, use aggregate instead
  mapreduce_info = db.passengers.map_reduce(mapfun,


In [5]:
mapreduce_info

{'result': 'survivedPassengers', 'ok': 1.0}

In [6]:
for doc in db.survivedPassengers.find():
    print(doc)

{'_id': 1.0, 'value': {'countMale': 36.89, 'countFemale': 96.81}}
{'_id': 2.0, 'value': {'countMale': 15.74, 'countFemale': 92.11}}
{'_id': 3.0, 'value': {'countMale': 13.54, 'countFemale': 50.0}}


In [7]:
# La semplice importazione del risultato della query a MongoDB
# ci restituisce un dataframe che non ha le caratteristiche volute

pd.DataFrame(db.survivedPassengers.find())

,_id,value
0,1.0,"{'countMale': 36.89, 'countFemale': 96.81}"
1,2.0,"{'countMale': 15.74, 'countFemale': 92.11}"
2,3.0,"{'countMale': 13.54, 'countFemale': 50.0}"


In [8]:
raw_data = list(db.survivedPassengers.find())

transformed_data = []

for data in raw_data:
    transformed_data.append({'Class': data['_id'],
                             'Male %': data['value']['countMale'],
                             'Female %': data['value']['countFemale']})

survived_passengers = pd.DataFrame(transformed_data)
survived_passengers.set_index('Class')

,Male %,Female %
Class,,
1.0,36.89,96.81
2.0,15.74,92.11
3.0,13.54,50.00
